In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import time

device = torch.device('mps')

In [2]:
# indirince bunu bi denerik
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [63]:
train_transforms = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.RandAugment(),
    transforms.ToTensor(),
])
test_transforms = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
])
# Create Training dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=train_transforms, download=True)

# Create Testing dataset
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=test_transforms, download=True)

batch_size = 128
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [64]:
# Define the CNN model
class AlexNet(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        self.features= nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(256, 384, (3, 3), stride=1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(384, 384, (3, 3), stride=1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(384, 256, (3, 3), stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d((3, 3), (2, 2)),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Linear(4096, num_classes),
        )

    def print_sizes(self, model, input_tensor):
        output = input_tensor
        for m in model.children():
            print(m, output.shape)
            output = m(output)
        return output
    def forward(self, x):
        # self.print_sizes(self.features, x)
        out = self.features(x)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out

In [65]:
num_classes = 10
learning_rate = 0.001
num_epochs = 20


model = AlexNet(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0005)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    loss = None
    t = time.time()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    correct = 0
    with torch.no_grad():
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {}, Elapsed: {} '.format(epoch + 1, num_epochs, loss.item(),
                                                                           100 * correct / total, time.time() - t))

Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4)) torch.Size([128, 3, 227, 227])
ReLU(inplace=True) torch.Size([128, 96, 55, 55])
MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False) torch.Size([128, 96, 55, 55])
Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)) torch.Size([128, 96, 27, 27])
ReLU(inplace=True) torch.Size([128, 256, 27, 27])
MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False) torch.Size([128, 256, 27, 27])
Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) torch.Size([128, 256, 13, 13])
ReLU(inplace=True) torch.Size([128, 384, 13, 13])
Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) torch.Size([128, 384, 13, 13])
ReLU(inplace=True) torch.Size([128, 384, 13, 13])
Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) torch.Size([128, 384, 13, 13])
ReLU(inplace=True) torch.Size([128, 256, 13, 13])
MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation

KeyboardInterrupt: 

In [6]:
"""
Epoch [1/20], Loss: 1.3613, Accuracy: 53.62, Elapsed: 15.32989501953125 
Epoch [2/20], Loss: 0.8447, Accuracy: 63.2, Elapsed: 15.123994827270508 
Epoch [3/20], Loss: 0.8678, Accuracy: 65.93, Elapsed: 15.185999870300293 
Epoch [4/20], Loss: 0.8962, Accuracy: 68.71, Elapsed: 15.104533195495605 
Epoch [5/20], Loss: 0.5207, Accuracy: 70.58, Elapsed: 15.10148310661316 
Epoch [6/20], Loss: 0.6476, Accuracy: 70.49, Elapsed: 15.253604650497437 
Epoch [7/20], Loss: 0.5595, Accuracy: 70.84, Elapsed: 15.609240055084229 
Epoch [8/20], Loss: 0.5878, Accuracy: 70.2, Elapsed: 15.391446113586426 
Epoch [9/20], Loss: 0.4980, Accuracy: 69.8, Elapsed: 15.125716209411621 
Epoch [10/20], Loss: 0.2939, Accuracy: 69.97, Elapsed: 15.214236974716187 
Epoch [11/20], Loss: 0.3107, Accuracy: 70.41, Elapsed: 15.064754962921143 
Epoch [12/20], Loss: 0.3099, Accuracy: 70.17, Elapsed: 15.302250146865845 
Epoch [13/20], Loss: 0.3888, Accuracy: 68.96, Elapsed: 15.270372152328491 
Epoch [14/20], Loss: 0.3394, Accuracy: 69.54, Elapsed: 15.145703077316284 
Epoch [15/20], Loss: 0.2791, Accuracy: 69.18, Elapsed: 15.143717050552368
"""

'\nEpoch [1/20], Loss: 1.3613, Accuracy: 53.62, Elapsed: 15.32989501953125 \nEpoch [2/20], Loss: 0.8447, Accuracy: 63.2, Elapsed: 15.123994827270508 \nEpoch [3/20], Loss: 0.8678, Accuracy: 65.93, Elapsed: 15.185999870300293 \nEpoch [4/20], Loss: 0.8962, Accuracy: 68.71, Elapsed: 15.104533195495605 \nEpoch [5/20], Loss: 0.5207, Accuracy: 70.58, Elapsed: 15.10148310661316 \nEpoch [6/20], Loss: 0.6476, Accuracy: 70.49, Elapsed: 15.253604650497437 \nEpoch [7/20], Loss: 0.5595, Accuracy: 70.84, Elapsed: 15.609240055084229 \nEpoch [8/20], Loss: 0.5878, Accuracy: 70.2, Elapsed: 15.391446113586426 \nEpoch [9/20], Loss: 0.4980, Accuracy: 69.8, Elapsed: 15.125716209411621 \nEpoch [10/20], Loss: 0.2939, Accuracy: 69.97, Elapsed: 15.214236974716187 \nEpoch [11/20], Loss: 0.3107, Accuracy: 70.41, Elapsed: 15.064754962921143 \nEpoch [12/20], Loss: 0.3099, Accuracy: 70.17, Elapsed: 15.302250146865845 \nEpoch [13/20], Loss: 0.3888, Accuracy: 68.96, Elapsed: 15.270372152328491 \nEpoch [14/20], Loss: 0.